In [1]:
!pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 325 kB 28.9 MB/s 
     |████████████████████████████████| 4.0 MB 60.8 MB/s 
     |████████████████████████████████| 1.1 MB 31.6 MB/s 
     |████████████████████████████████| 212 kB 75.2 MB/s 
     |████████████████████████████████| 77 kB 8.5 MB/s 
     |████████████████████████████████| 136 kB 78.5 MB/s 
     |████████████████████████████████| 127 kB 74.5 MB/s 
     |████████████████████████████████| 271 kB 70.5 MB/s 
     |████████████████████████████████| 144 kB 73.8 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 895 kB 61.0 MB/s 
     |████████████████████████████████| 6.6 MB 38.3 MB/s 
     |████████████████████████████████| 596 kB 58.8 MB/s 
     |████████████████████████████████| 1.2 MB 43.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
  

In [2]:
from transformers import AutoModelForSequenceClassification, XLMRobertaForSequenceClassification
from transformers import AdamW
from transformers import get_scheduler
import torch
from tqdm.auto import tqdm
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding, XLMRobertaConfig, PretrainedConfig, AutoConfig

from torch.utils.data import DataLoader
import random
import numpy

numpy.random.seed(0)
random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False

In [3]:
check_poin = "xlm-roberta-base"
config = AutoConfig.from_pretrained(check_poin)
model =  AutoModelForSequenceClassification.from_config(config)
tokenizer = AutoTokenizer.from_pretrained(check_poin)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [4]:
dataset = load_dataset('csv', data_files="/content/drive/MyDrive/Long/sentiment-analysis/train.csv", split='train')
dataset = dataset.train_test_split(shuffle = True, seed = 200, test_size=0.3)

data_without_label = load_dataset('csv', data_files="/content/drive/MyDrive/Long/sentiment-analysis/train_without_label.csv")
data_without_label['train'] = data_without_label['train'].select(range(2000))
dataset

Using custom data configuration default-19630b4e12c7a440


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-19630b4e12c7a440/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


Using custom data configuration default-f17b45b4ec575a8b


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f17b45b4ec575a8b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Content', 'Label'],
        num_rows: 5630
    })
    test: Dataset({
        features: ['Content', 'Label'],
        num_rows: 2413
    })
})

In [5]:
def tokenize_function(example):
  return tokenizer(example["Content"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_without_label = data_without_label.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [6]:
tokenized_datasets = tokenized_datasets.rename_column('Label', 'label')
tokenized_datasets = tokenized_datasets.remove_columns(['Content'])
data_without_label = data_without_label.remove_columns(['Content'])
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5630
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 2413
    })
})

In [7]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=4, collate_fn=data_collator,
    worker_init_fn=seed_worker,
    generator=g,
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=4, collate_fn=data_collator,
    worker_init_fn=seed_worker,
    generator=g,
)

no_label_dataloader = DataLoader(
    data_without_label["train"], batch_size=4, collate_fn=data_collator,
    worker_init_fn=seed_worker,
    generator=g,
)


optimizer = AdamW(model.parameters(), lr=5e-6)
num_epochs = 10
num_training_steps = num_epochs * len(no_label_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(len(no_label_dataloader))

500


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [8]:
from datasets import load_metric
T1 = 100
T2 = 700
af = 3
metric = load_metric("accuracy")

def evaluate(eval_dataloader):
  model.eval()
  for batch in eval_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      with torch.no_grad():
          outputs = model(**batch)

      logits = outputs.logits
      predictions = torch.argmax(logits, dim=-1)
      metric.add_batch(predictions=predictions, references=batch["labels"])

  res = metric.compute()
  return res

def alpha_weight(step):
    if step < T1:
        return 0.0
    elif step > T2:
        return af
    else:
         return ((step-T1) / (T2-T1))*af

In [ ]:
progress_bar = tqdm(range(num_training_steps))

step = 100
for epoch in range(num_epochs):
  print(f"Epoch {epoch}")
  for batch_id, batch in enumerate(no_label_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    model.eval()
    outputs = model(**batch)
    logits = outputs.logits
    labels = torch.argmax(logits, dim=-1)
    batch['labels'] = labels
    model.train()
    outputs = model(**batch)
    loss = alpha_weight(step) * outputs.loss

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    lr_scheduler.step()

    if batch_id % 50 == 0:
      for batch in train_dataloader:
          batch = {k: v.to(device) for k, v in batch.items()}
          outputs = model(**batch)
          labeled_loss = outputs.loss
          labeled_loss.backward()
          optimizer.step()
          optimizer.zero_grad()
      res = evaluate(eval_dataloader)
      res['loss'] = loss.item()
      print(res)
      step += 1
    progress_bar.update(1)
  


  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 0
{'accuracy': 0.7845006216328222, 'loss': 0.0}
{'accuracy': 0.8690426854537919, 'loss': 0.000547399977222085}
{'accuracy': 0.8027351844177373, 'loss': 0.0015184709336608648}
{'accuracy': 0.8781599668462495, 'loss': 0.0004455371235962957}
{'accuracy': 0.8723580605055947, 'loss': 0.001998684136196971}
{'accuracy': 0.8856195607128057, 'loss': 0.0002075240045087412}
{'accuracy': 0.8814753418980522, 'loss': 0.0022486578673124313}
{'accuracy': 0.8677994198093659, 'loss': 0.0010116971097886562}
{'accuracy': 0.8814753418980522, 'loss': 0.00032051256857812405}
{'accuracy': 0.8727724823870701, 'loss': 0.0011546466266736388}
Epoch 1
{'accuracy': 0.8702859510982179, 'loss': 0.0002571784134488553}
{'accuracy': 0.8765022793203481, 'loss': 0.0017585562309250236}
{'accuracy': 0.8727724823870701, 'loss': 0.0003530423273332417}
{'accuracy': 0.8760878574388727, 'loss': 0.0013140375958755612}
{'accuracy': 0.8765022793203481, 'loss': 0.0013176400680094957}
{'accuracy': 0.8694571073352673, 'loss': 0.

In [ ]:
evaluate(eval_dataloader)